In [1]:
import dask.dataframe as dd
import numpy as np
import os
import pandas as pd

#### Thay đổi hiển thị ####
pd.reset_option('display.max_columns')
pd.reset_option('display.max_rows')
pd.set_option('display.max_row', None)
pd.set_option('display.max_columns', None)
#### Change display ####

input_file = ["/mnt/c/Users/hoang/FileCSV_DACN_2025/shuffle_IoT23.csv", "C:\\Users\\hoang\\FileCSV_DACN_2025\\shuffle_IoT23.csv"]

if os.name == 'nt':
    input_file = input_file[1]
else:
    input_file = input_file[0]

df = dd.read_csv(input_file)
# dtype={'detailed-label': 'object'}, assume_missing=True

# print(df.columns)
dictTypes = {}
df = dd.read_csv(input_file)
for col in df.columns:
    if col.startswith('proto') == True:
        dictTypes[col] = 'int32'
    elif col.startswith('service_') == True:
        dictTypes[col] = 'int32'
    # elif col == 'label':
    #     dictTypes[col]= 'int32'
    elif col.startswith('detailed-label'):
        dictTypes[col] = 'int32'
    else:
        dictTypes[col]='float32'

df = dd.read_csv(input_file, dtype = dictTypes)
df = df.drop(columns=['label'])

# phân loại đa nhãn, k dùng
# df = df.replace(r'[N|n][a|A][N|n]', 0)
# df = df.replace(np.nan, 0)
print(df.dtypes)
# print(df['duration'].value_counts().compute())
print(df.tail())

id.orig_h         float32
id.orig_p         float32
id.resp_h         float32
id.resp_p         float32
duration          float32
orig_bytes        float32
resp_bytes        float32
conn_state        float32
missed_bytes      float32
history           float32
orig_pkts         float32
orig_ip_bytes     float32
resp_pkts         float32
resp_ip_bytes     float32
detailed-label      int32
proto_icmp          int32
proto_tcp           int32
proto_udp           int32
service_dhcp        int32
service_dns         int32
service_http        int32
service_irc         int32
service_ssh         int32
service_ssl         int32
dtype: object
        id.orig_h  id.orig_p  id.resp_h  id.resp_p  duration  orig_bytes  \
547395   3.232236   0.686359   1.529949   0.550982       0.0         0.0   
547396   3.232236   0.713139   3.111203   0.550982       0.0         0.0   
547397   3.232236   0.705803   3.119152   0.550982       0.0         0.0   
547398   3.232236   0.500587   0.759364   0.550982       0

In [14]:
label_counts = df['detailed-label'].value_counts().compute()

In [ ]:
print(label_counts)

detailed-label
Attack                                   9398
Okiru-Attack                                3
C&C-HeartBeat-Attack                      834
PartOfAHorizontalPortScan-Attack            5
FileDownload                               18
C&C-Mirai                                   2
DDoS                                 16108989
C&C-FileDownload                           53
Okiru                                60938340
C&C-HeartBeat                           31528
C&C-HeartBeat-FileDownload                 11
C&C                                     17657
PartOfAHorizontalPortScan           213583302
C&C-Torii                                  23
0                                    30771095
C&C-PartOfAHorizontalPortScan             797
Name: count, dtype: int64[pyarrow]


In [ ]:

import matplotlib.pyplot as plt
ordered_labels = [0, 1, 2, 3]
ordered_counts = [label_counts.get(label, 0) for label in ordered_labels] 
print(label_counts)
labels = ["Benign","DDoS","DoS","Mirai"]
# Vẽ biểu đồ cột
plt.figure(figsize=(9, 5))
plt.bar(labels, ordered_counts, color='skyblue', edgecolor='black')
plt.xlabel("Nhãn (Classes)")
plt.ylabel("Số lượng mẫu (Frequency)")
plt.title("Tỷ lệ nhãn trong Dataset")
plt.xticks(range(len(labels)) ,labels, rotation =45)
plt.show()

In [4]:
df = df[df['detailed-label'].isin(['Okiru', 'PartOfAHorizontalPortScan', '0'])]

label_map = {
    '0': 0,
    'PartOfAHorizontalPortScan': 1,
    'Okiru': 2
}

df['detailed-label'] = df['detailed-label'].map(label_map).astype('int32')

/home/hoangvn/miniconda3/miniconda3/envs/doan/lib/python3.9/site-packages/dask_expr/_collection.py:4190: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('detailed-label', 'float64'))

  warnings.warn(meta_warning(meta))


In [2]:
print(df.dtypes)

id.orig_h         float32
id.orig_p         float32
id.resp_h         float32
id.resp_p         float32
duration          float32
orig_bytes        float32
resp_bytes        float32
conn_state        float32
missed_bytes      float32
history           float32
orig_pkts         float32
orig_ip_bytes     float32
resp_pkts         float32
resp_ip_bytes     float32
detailed-label      int32
proto_icmp          int32
proto_tcp           int32
proto_udp           int32
service_dhcp        int32
service_dns         int32
service_http        int32
service_irc         int32
service_ssh         int32
service_ssl         int32
dtype: object


# Generator + Train

In [2]:
import numpy as np
import pandas as pd
import dask.dataframe as dk
import tensorflow as tf 
from tensorflow.keras.utils import Sequence, to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from datetime import datetime, timedelta
from tensorflow import keras

#Global var 
batch_size = 512
ratio_test_all = 0.2

from dask_ml.model_selection import train_test_split 
# Bước 1: Tách 80% train, 20% còn lại (val + test)
train_df, val_test_df = train_test_split(df, test_size=0.20, random_state=42)
val_df, test_df = train_test_split(val_test_df, test_size=0.75, random_state=42)

/home/hoangvn/miniconda3/miniconda3/envs/doan/lib/python3.9/site-packages/dask_ml/model_selection/_split.py:464: FutureWarning: The default value for 'shuffle' must be specified when splitting DataFrames. In the future DataFrames will automatically be shuffled within blocks prior to splitting. Specify 'shuffle=True' to adopt the future behavior now, or 'shuffle=False' to retain the previous behavior.
  warnings.warn(
/home/hoangvn/miniconda3/miniconda3/envs/doan/lib/python3.9/site-packages/dask_ml/model_selection/_split.py:464: FutureWarning: The default value for 'shuffle' must be specified when splitting DataFrames. In the future DataFrames will automatically be shuffled within blocks prior to splitting. Specify 'shuffle=True' to adopt the future behavior now, or 'shuffle=False' to retain the previous behavior.
  warnings.warn(


In [3]:
features_len = len(df.columns)-1
print("Feature Len: ",features_len)
def dask_to_tf_dataset(dask_df, batch_size, num_classes): 
    def generator():
        for batch in dask_df.to_delayed():
            batch=batch.compute()  
            if batch.empty:
                continue

            X = batch.drop(columns='detailed-label').values.astype(np.float32)
            y = batch['detailed-label'].values
            y_onehot = to_categorical(y, num_classes=num_classes)  

            num_splits = max(1, len(X) // batch_size)  # Đảm bảo không chia nhỏ quá mức
            X_batches = np.array_split(X, num_splits)
            y_batches = np.array_split(y_onehot, num_splits)

            for X_batch, y_batch in zip(X_batches, y_batches):
                yield X_batch, y_batch
                
    output_signature = ( 
        tf.TensorSpec(shape=(None, features_len), dtype=tf.float32), 
        tf.TensorSpec(shape=(None, 3), dtype=tf.int32),
    )
    
    return tf.data.Dataset.from_generator(generator, output_signature=output_signature).prefetch(tf.data.AUTOTUNE)

Feature Len:  23


In [4]:
batchSize = 512
train_gen = dask_to_tf_dataset(train_df, batchSize, 3).repeat()
val_gen = dask_to_tf_dataset(val_df, batchSize, 3).repeat()
test_gen = dask_to_tf_dataset(test_df, batchSize, 3)

I0000 00:00:1747845466.516584   51638 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2248 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650 with Max-Q Design, pci bus id: 0000:02:00.0, compute capability: 7.5


In [5]:
import math
n_samples =  np.ceil(train_df.shape[0])
steps_per_epoch = int(n_samples / (batchSize))
validation_steps = int(steps_per_epoch / (16))
print("Steps/Epoch: ", steps_per_epoch)

Steps/Epoch:  133157


In [7]:
########### Enable XLA ###############
tf.config.optimizer.set_jit(True)

########### Nếu không dùng XLA ###########
# import os
# os.environ["TF_XLA_FLAGS"] = "--tf_xla_auto_jit=0"

# shape
features, labels = next(iter(train_gen))
input_shape = (features.shape[1], 1)
output_shape = labels.shape[1]

print(f"Input Shape: {input_shape}")

# Định nghĩa mô hình CNN
# VGG, ...
# Conv2D, tabular, ...
# HE, tính tương thích của HE với CNN
# Tính chất data in, out; Học tăng cường
start_time = datetime.now()

model = keras.Sequential([
    layers.Input(shape=input_shape),
    layers.Conv1D(filters=128, kernel_size=7, padding="same", activation="relu"),
    layers.BatchNormalization(),
    layers.Conv1D(filters=128, kernel_size=7,  padding="same",activation="relu"),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Dropout(0.1),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(output_shape, activation='softmax')
])
adam_optimizer = optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=adam_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
#sparse khi không onehot
# for batch in dataloader:
#     X_batch = batch[:, :-1]
#     y_batch = batch[:, -1]
#     y_onehot = to_categorical(y_batch, num_classes=10)
    
#     model.train_on_batch(X_batch, y_onehot, verbose=1)
from tensorflow.keras.callbacks import CSVLogger

csv_logger = CSVLogger("Centralized_Log/"+ datetime.now().strftime("Month %m Day %d __ %Hh--%Mp")+".csv" , append=True)
model.fit(train_gen, epochs=10,  validation_data=val_gen, 
          validation_steps=validation_steps, steps_per_epoch=int(steps_per_epoch/10), verbose = 1, callbacks=[csv_logger])

end_time = datetime.now()
simulated_time = end_time - start_time

# Lưu mô hình
model.save("Centralized_Model/cnn_model_" + datetime.now().strftime("Month %m Day %d __ %Hh--%Mp")+".keras")

print(f"Simulated time: {simulated_time}")

Input Shape: (23, 1)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 23, 128)        │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 23, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 23, 128)        │       114,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 23, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 11, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 11, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1408)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       180,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 298,115 (1.14 MB)

 Trainable params: 297,347 (1.13 MB)

 Non-trainable params: 768 (3.00 KB)

Epoch 1/10


I0000 00:00:1747845782.271419   63802 service.cc:152] XLA service 0x7ff5180049b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747845782.272168   63802 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce GTX 1650 with Max-Q Design, Compute Capability 7.5
I0000 00:00:1747845782.339617   63802 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1747845782.556028   63802 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2025-05-21 16:43:05.321363: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


13315/13315 ━━━━━━━━━━━━━━━━━━━━ 422s 31ms/step - accuracy: 0.9160 - loss: 0.3965 - val_accuracy: 0.2013 - val_loss: 21.2993
Epoch 2/10
13315/13315 ━━━━━━━━━━━━━━━━━━━━ 396s 30ms/step - accuracy: 0.9234 - loss: 0.3957 - val_accuracy: 0.3971 - val_loss: 29.1322
Epoch 3/10
13315/13315 ━━━━━━━━━━━━━━━━━━━━ 391s 29ms/step - accuracy: 0.9384 - loss: 0.3615 - val_accuracy: 0.6428 - val_loss: 43.3081
Epoch 4/10
13315/13315 ━━━━━━━━━━━━━━━━━━━━ 369s 28ms/step - accuracy: 0.9375 - loss: 0.3391 - val_accuracy: 0.7005 - val_loss: 1.4298
Epoch 5/10
13315/13315 ━━━━━━━━━━━━━━━━━━━━ 419s 31ms/step - accuracy: 0.9492 - loss: 0.3458 - val_accuracy: 0.5008 - val_loss: 685.8297
Epoch 6/10
13311/13315 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9580 - loss: 0.2972

: 

In [4]:
batchSize = 512
n_partitions = train_df.npartitions  # Tổng số chunk hiện có
print(n_partitions)

# # Tạo validation dataset cố định 1 lần
# val_df_pd = val_df.compute()
# X_val = val_df_pd.drop(columns='detailed-label').values.astype(np.float32)
# y_val = tf.keras.utils.to_categorical(val_df_pd['detailed-label'].values, num_classes=3)

# del val_df_pd
# val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
# del X_val
# del y_val
# val_dataset = val_dataset.batch(batchSize).prefetch(tf.data.AUTOTUNE)

def df_partition_to_dataset(dask_partition, num_classes=3, batch_size=512):
    df = dask_partition.compute()
    if df.empty:
        return None
    X = df.drop(columns='detailed-label').values.astype(np.float32)
    y = tf.keras.utils.to_categorical(df['detailed-label'].values, num_classes)
    ds = tf.data.Dataset.from_tensor_slices((X, y))
    return ds.shuffle(len(X)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

72


In [ ]:
start_time = datetime.now()


model = keras.Sequential([
    layers.Input(shape=(23,1)),
    layers.Conv1D(filters=128, kernel_size=7, padding="same", activation="relu"),
    layers.BatchNormalization(),
    layers.Conv1D(filters=128, kernel_size=7,  padding="same",activation="relu"),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Dropout(0.1),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train lần lượt trên từng 
from tensorflow.keras.callbacks import CSVLogger
csv_logger = CSVLogger("Centralized_Log/"+ datetime.now().strftime("Month %m Day %d __ %Hh%Mp")+".csv" , append=True)
nepochs=25
for epoch in range(nepochs):
    for i in range(n_partitions):
        print(f"\n🔁 Training on partition {i+1}/{n_partitions}")
        train_partition = train_df.get_partition(i)
        val_partition = val_df.get_partition(i)
        train_ds = df_partition_to_dataset(train_partition, 3, 128)
        val_ds = df_partition_to_dataset(val_partition, 3, 128)
        if train_ds is not None:
            model.fit(train_ds, epochs=1,  validation_data=val_ds, verbose = 1, callbacks=[csv_logger])
            
end_time = datetime.now()
simulated_time = end_time - start_time

# Lưu mô hình
model.save("Centralized_Model/cnn_model_" + datetime.now().strftime("Month %m Day %d __ %Hh%Mp")+".keras")


🔁 Training on partition 1/72


I0000 00:00:1746379085.743877   39304 service.cc:152] XLA service 0x7f08b4010210 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746379085.743948   39304 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce GTX 1650 with Max-Q Design, Compute Capability 7.5
2025-05-04 17:18:05.836701: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1746379086.198713   39304 cuda_dnn.cc:529] Loaded cuDNN version 90300


   31/20147 ━━━━━━━━━━━━━━━━━━━━ 1:44 5ms/step - accuracy: 0.8444 - loss: 0.4347

I0000 00:00:1746379089.397428   39304 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


20147/20147 ━━━━━━━━━━━━━━━━━━━━ 127s 6ms/step - accuracy: 0.9902 - loss: 0.0401 - val_accuracy: 0.9927 - val_loss: 0.0716

🔁 Training on partition 2/72
21419/21419 ━━━━━━━━━━━━━━━━━━━━ 134s 6ms/step - accuracy: 1.0000 - loss: 2.6914e-04 - val_accuracy: 1.0000 - val_loss: 1.3734e-04

🔁 Training on partition 3/72


: 